In [15]:
import pandas as pd
from functools import reduce
import os

# Define country codes
countries = ["CZ", "HUN", "PL", "SK"]

# Empty list to store country dataframes
dfs = []

for country in countries:
    # Construct the file path
    file_path = os.path.join("../data/raw", f"{country}_.csv")
   
    # Read CSV
    df = pd.read_csv(file_path)

    # Assume the first column is the date or month
    date_col = df.columns[6]
    value_col = df.columns[7]
    filter_col = df.columns[4]  
    
    # Filter by inflation type
    inflation = df[df[filter_col] == "All-items HICP"].copy()
    core_inflation = df[df[filter_col] == "Overall index excluding energy, food, alcohol and tobacco"].copy()
    
    # Make sure both have same date range
    inflation.reset_index(drop=True, inplace=True)
    core_inflation.reset_index(drop=True, inplace=True)
    
    # Rename value columns to indicate country and type
    inflation.rename(columns={value_col: f"inflation_{country}"}, inplace=True)
    core_inflation.rename(columns={value_col: f"core_inflation_{country}"}, inplace=True)
    
    #Merge inflation and core inflatuion for given country
    merged = pd.merge(
        inflation[[date_col, f"inflation_{country}"]],
        core_inflation[[date_col, f"core_inflation_{country}"]],
        on=date_col,
        how="outer"
    )
    
    dfs.append(merged)

# Combine all countries
combined = reduce(lambda i, j: pd.merge(i, j, on=date_col, how="outer"), dfs)
combined.sort_values(by=date_col, inplace=True)
  
print(combined.head())

#Save data frame
combined.to_csv("../data/combined_inflation_data.csv", index=False)

print("✅ Combined dataset saved as 'combined_inflation_data.csv'")


  TIME_PERIOD  inflation_CZ  core_inflation_CZ  inflation_HUN  \
0     2015-01          -0.1                0.8           -1.4   
1     2015-02           0.0                0.7           -0.9   
2     2015-03           0.1                0.7           -0.5   
3     2015-04           0.5                0.7            0.0   
4     2015-05           0.7                0.9            0.6   

   core_inflation_HUN  inflation_PL  core_inflation_PL  inflation_SK  \
0                 1.2          -1.1                0.5          -0.5   
1                 1.5          -1.3                0.5          -0.6   
2                 1.5          -1.2                0.3          -0.4   
3                 1.8          -0.8                0.4          -0.1   
4                 1.9          -0.6                0.5          -0.1   

   core_inflation_SK  
0                0.5  
1                0.5  
2                0.5  
3                0.4  
4                0.5  


C:\Users\borik\Desktop\inflation_in_V4\code
